In [2]:
import pandas as pd
import lightgbm as lgb
import sklearn
import gplearn
import numpy as np
import gc
import os

In [3]:
from gplearn.genetic import SymbolicTransformer

In [4]:
Path = '/home/kai/data/kaggle/talkingdata/4thplace/'

In [5]:
train_original = pd.read_feather(Path + '0.ftr')

In [6]:
train_original.columns.values

array(['index', 'ip', 'app', 'device', 'os', 'channel', 'click_time',
       'is_attributed', 'click_id', 'is_test', 'day', 'hour', 'minute',
       'minute10', 'hourminute', 'hourminute10', 'dayhourminute',
       'dayhourminute10', 'machine', 'click_timestamp',
       'woe_ip_app_channel', 'woe_ip_device_nextClickLeakDayFlt',
       'nunique_machine_app', 'woe_os_channel_nextClickLeakDayFlt',
       'variance_ip_device_hour',
       'range_count_ip_device_os_dayhourminute10',
       'woe_app_device_os_channel', 'woe_channel',
       'count_ip_app_os_channel', 'nunique_day_ip_os', 'count_app_channel',
       'woe_app_nextClickLeakDayFlt', 'woe_ip_channel',
       'unique_ratio_day_ip_machine', 'nunique_machine_channel',
       'time_to_n_next_click_1_day_ip_app_device_os', 'woe_os',
       'woe_ip_device_os_channel', 'count_ip_app_device_day', 'com1_ip',
       'unique_ratio_machine_ip', 'count_ratio_ip_machine',
       'count_ip_app_device_os_day_hour', 'count_app_device_day_hour',
 

In [7]:
print(train_original.shape)
# train = pd.concat([train_original, val_original], axis = 0)

(794952, 118)


In [8]:
train_original.columns.values

array(['index', 'ip', 'app', 'device', 'os', 'channel', 'click_time',
       'is_attributed', 'click_id', 'is_test', 'day', 'hour', 'minute',
       'minute10', 'hourminute', 'hourminute10', 'dayhourminute',
       'dayhourminute10', 'machine', 'click_timestamp',
       'woe_ip_app_channel', 'woe_ip_device_nextClickLeakDayFlt',
       'nunique_machine_app', 'woe_os_channel_nextClickLeakDayFlt',
       'variance_ip_device_hour',
       'range_count_ip_device_os_dayhourminute10',
       'woe_app_device_os_channel', 'woe_channel',
       'count_ip_app_os_channel', 'nunique_day_ip_os', 'count_app_channel',
       'woe_app_nextClickLeakDayFlt', 'woe_ip_channel',
       'unique_ratio_day_ip_machine', 'nunique_machine_channel',
       'time_to_n_next_click_1_day_ip_app_device_os', 'woe_os',
       'woe_ip_device_os_channel', 'count_ip_app_device_day', 'com1_ip',
       'unique_ratio_machine_ip', 'count_ratio_ip_machine',
       'count_ip_app_device_os_day_hour', 'count_app_device_day_hour',
 

In [9]:
# cols = ['woe_app_channel_nextClickLeakDayFlt', 
#         'woe_app_os_channel', 
#         'woe_app_os_nextClickLeakDayFlt',
#         'woe_app_device_os_channel',
#         'woe_app_nextClickLeakDayFlt',
#         'woe_app_channel',
#         'woe_app_device_os'
#        ]
cols = ['cumcount_ratio_ip_day', 'variance_ip_device_hour', 'woe_app_channel_nextClickLeakDayFlt', 'woe_app_os_channel', 'nunique_day_ip_machine', 'nunique_day_ip_app','woe_app_os_nextClickLeakDayFlt','woe_app_device_os_channel','woe_app_nextClickLeakDayFlt','time_to_n_next_click_1_day_ip_app_device_os']

In [10]:
# cols = []

In [11]:
# Get target col
target_array = train_original['is_attributed'].values

In [12]:
train_array = train_original[cols].values

In [15]:
train_correlations = train_original.corr(method = 'spearman')['is_attributed'].sort_values()

In [30]:
x = train_correlations.abs().sort_values()
print(x.values)
print(list(x.index[:87]))


[ 0.00449252  0.00550383  0.00563494  0.00675875  0.00692652  0.00693784
  0.00698565  0.00698618  0.00698618  0.00698878  0.00979151  0.06262207
  0.07700649  0.09096568  0.12727049  0.16154448  0.18334107  0.19582774
  0.23008546  0.23778724  0.24855884  0.25503288  0.26819794  0.27121131
  0.28377899  0.28733673  0.28901124  0.28950139  0.28968914  0.28999994
  0.29214694  0.29332229  0.30405961  0.30551245  0.30606593  0.31971443
  0.32011775  0.32297864  0.32819927  0.32897225  0.33195352  0.33975505
  0.35233788  0.35382058  0.35663938  0.35700275  0.36303022  0.36553421
  0.37589123  0.37805224  0.38088304  0.40766476  0.40817879  0.40967886
  0.41857207  0.43586337  0.43769233  0.43776251  0.43790846  0.43863981
  0.43919278  0.440426    0.44340296  0.44435521  0.44504849  0.44544413
  0.44614908  0.44769662  0.45106853  0.45296649  0.45420405  0.4542418
  0.45516454  0.45525692  0.4605018   0.4618708   0.46584735  0.47249273
  0.47736612  0.48617828  0.49516238  0.50401706  0.

In [37]:
cols_shrink = list(x.index[87:-3])
print(index_shrink)

['range_count_ip_channel_dayhourminute10', 'range_count_ip_channel_dayhourminute', 'time_to_n_next_click_1_day_ip_app_device_os', 'count_ip_app_device_channel_day', 'woe_device_os_nextClickLeakDayFlt', 'woe_os_nextClickLeakDayFlt', 'count_app_channel', 'count_app_device_channel_day_hour', 'count_app_device_day_hour', 'count_app_day_hourminute', 'count_ip_app_device_day', 'woe_app_device_os_channel', 'woe_app_device_os', 'woe_app_device', 'woe_channel', 'woe_app_device_channel', 'woe_app_os_channel', 'woe_app_os', 'woe_app', 'woe_device_channel_nextClickLeakDayFlt', 'woe_app_channel', 'woe_app_device_nextClickLeakDayFlt', 'woe_os_channel_nextClickLeakDayFlt', 'woe_channel_nextClickLeakDayFlt', 'woe_app_os_nextClickLeakDayFlt', 'woe_app_nextClickLeakDayFlt', 'woe_app_channel_nextClickLeakDayFlt']


In [38]:
train_shrink = train_original[cols_shrink]
train_array_shrink = train_shrink.values

In [39]:
# function_set = ['add','sub', 'mul','div','sqrt', 'log', 'abs', 'neg', 'inv', 'max', 'min']


function_set = ['add','sub', 'mul','div', 'abs', 'neg', 'max', 'min', 'sqrt', 'sin', 'cos', 'log', 'inv']
gp = SymbolicTransformer(generations=20,
                         population_size=2000, 
                         stopping_criteria = 1, 
                         tournament_size = 1000,
                         metric = 'spearman',
                         hall_of_fame=100, 
                         n_components=15,
                         function_set=function_set,
                         parsimony_coefficient=0.0005,
                         max_samples=1, 
                         verbose=1,
                         random_state=0,
                         n_jobs=30,
                        p_subtree_mutation=0.03,
                        p_hoist_mutation = 0.04,
                        p_point_mutation = 0.03)

# SymbolicTransformer(const_range=(-1.0, 1.0),
#           function_set=['add', 'sub', 'mul', 'div', 'abs', 'neg', 'max', 'min', 'sqrt', 'sin', 'cos', 'log', 'inv'],
#           generations=25, hall_of_fame=100, init_depth=(2, 6),
#           init_method='half and half', max_samples=0.9, metric='spearman',
#           n_components=10, n_jobs=20, p_crossover=0.9,
#           p_hoist_mutation=0.01, p_point_mutation=0.01,
#           p_point_replace=0.05, p_subtree_mutation=0.01,
#           parsimony_coefficient=0.0005, population_size=500,
#           random_state=0, stopping_criteria=0.95, tournament_size=20,
#           verbose=1, warm_start=False)

In [40]:
# %env JOBLIB_TEMP_FOLDER=/tmp
%env JOBLIB_TEMP_FOLDER=/home/kai/disksata/tmp
gp.fit(train_array, target_array)

env: JOBLIB_TEMP_FOLDER=/home/kai/disksata/tmp
    |    Population Average   |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     9.71   0.427829964061        5   0.832397440653              N/A     15.58m
   1     6.51   0.649282252598       15   0.851707786257              N/A     14.75m
   2    12.85   0.636197587575       28   0.856249515436              N/A     18.79m
   3    16.64     0.7355539497       26   0.856738081337              N/A     21.84m


Process ForkPoolWorker-143:


KeyboardInterrupt: 

In [16]:
gp_features_train = gp.transform(train_array)
print('train done')

In [17]:
train_index = train_original.index

In [18]:
train_new = pd.DataFrame(data = gp_features_train, index = train_index)
train = pd.concat([train_original, train_new], axis = 1)
print('train done')
val_original = pd.read_feather(Path + 'val_cleaned.ftr')
val_index = val.original.index
gp_features_val = gp_features_val = gp.transform(val_array)
val_new = pd.DataFrame(data = gp_features_val, index = val_index)
val = pd.concat([val_original, val_new], axis = 1)

train done


In [19]:
print(train.shape, val.shape)

(794952, 128) (22996974, 128)


In [20]:
val.to_pickle(Path + 'val_gp1.pkl')

In [21]:
train.to_pickle(Path + 'train_gp1.pkl')

In [23]:
test_supplement = pd.read_feather(Path + 'test_cleaned.ftr')
with open('./log.txt', 'a') as f: f.write('test rading done\n')
test_index = np.load('/home/kai/data/kaggle/talkingdata/data/supplement2testInexFinal.npy')
test = test_supplement.iloc[test_index]
print(test.shape)

(18790469, 118)


In [24]:
test_index = test.index

In [27]:
# print(test_index)
test_array = test[cols].values

In [28]:
gp_features_test = gp.transform(test_array)
test_new = pd.DataFrame(data = gp_features_test, index = test_index)
test = pd.concat([test, test_new], axis = 1)

In [29]:
test.to_pickle(Path + 'test_gp1.pkl')